In [202]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time

In [203]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)

In [204]:
class DynamicFcNN(nn.Module):
    def __init__(self, layers_count=3, activation_func=torch.relu):
        super(DynamicFcNN, self).__init__()
        if layers_count <= 0:
            raise Exception("layers_count must be greater than 0")
        self.activation_func, self.layers_count, self.flatten = activation_func, layers_count, nn.Flatten()
        in_size, layer_sizes = 28 * 28, []
        fc_i = 0
        while layers_count > 0:
            fc_i += 1
            layers_count -= 1
            out_size = 128 * (2 ** layers_count)
            setattr(self, f'fc{fc_i}', nn.Linear(in_size, out_size))
            layer_sizes.append([in_size, out_size])
            in_size = out_size
        self.fc_last = nn.Linear(out_size, 10)  # Выходной слой с 10 нейронами
        print('Создана модель с функцией активации %s, Слои: %s' % (activation_func.__name__, layer_sizes + [[out_size, 10]]))

    def forward(self, x):
        x = self.flatten(x)
        fc_i = 0
        while fc_i < self.layers_count:
            fc_i += 1
            layer = getattr(self, f'fc{fc_i}')
            x = self.activation_func(layer(x))
        return self.fc_last(x)

    def __repr__(self):
        return f"Количество слоев {self.layers_count}, Функция активации: '{self.activation_func.__name__}'"

In [205]:
def fc_learn(fc_model, step_size=0.1, num_epochs=5):
    print(f"Обучаем модель: {fc_model}")
    fc_criterion, fc_epoch_losses = nn.CrossEntropyLoss(), []
    fc_optimizer = optim.SGD(fc_model.parameters(), lr=step_size)
    fc_total_step = len(train_loader)
    fc_loss_list, fc_acc_list = [], []
    for fc_epoch in range(num_epochs): # num epochs
        fc_model.train()  # Установка модели в режим обучения
        fc_running_loss = 0.0
        for fc_i, (fc_images, fc_labels) in enumerate(train_loader):
            fc_optimizer.zero_grad()  # Обнуление градиентов
            fc_outputs = fc_model(fc_images)  # Получение выхода модели
            fc_loss = fc_criterion(fc_outputs, fc_labels)  # Вычисление потерь
            fc_loss.backward()  # Обратное распространение ошибки
            fc_optimizer.step()  # Обновление весов
            fc_running_loss += fc_loss.item() * fc_images.size(0)  # Учитываем потери по всем изображениям

            # Отслеживание точности
            fc_total = fc_labels.size(0)
            _, fc_predicted = torch.max(fc_outputs.data, 1)
            fc_correct = (fc_predicted == fc_labels).sum().item()
            fc_acc_list.append(fc_correct / fc_total)

            if (fc_i + 1) % 300 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                    .format(fc_epoch + 1, num_epochs, fc_i + 1, fc_total_step, fc_loss.item(), (fc_correct / fc_total) * 100))

        fc_epoch_losses.append(fc_running_loss / len(train_loader.dataset))  # Вычисляем среднюю потерю за эпоху
    print(f"Потери по эпохам:", fc_epoch_losses)

In [212]:
fc_model = DynamicFcNN(layers_count=2, activation_func=torch.relu)
start_time = time.time()
fc_learn(fc_model, step_size=0.05, num_epochs=1)
end_time = time.time()
print("Время обучения FC модели: {:.2f} секунд".format(end_time - start_time))

Создана модель с функцией активации relu, Слои: [[784, 256], [256, 128], [128, 10]]
Обучаем модель: Количество слоев 2, Функция активации: 'relu'
Epoch [1/1], Step [300/600], Loss: 0.4290, Accuracy: 89.00%
Epoch [1/1], Step [600/600], Loss: 0.2742, Accuracy: 91.00%
Потери по эпохам: [0.44562239915132523]
Время обучения FC модели: 24.92 секунд


In [214]:
class ConvNet(nn.Module):
    def __init__(self, layers_count=3, activation_func=nn.ReLU()):
        super(ConvNet, self).__init__()
        if layers_count <= 0:
            raise Exception("layers_count must be greater than 0")
        self.layers_count, self.drop_out = layers_count, nn.Dropout()
        in_size, out_size = 1, 32
        idx = 0
        current_features = 700
        while idx < self.layers_count:
            idx += 1
            layer = nn.Sequential(nn.Conv2d(in_size, out_size, kernel_size=5, stride=1, padding=2), activation_func, nn.MaxPool2d(kernel_size=2, stride=2))
            in_size, out_size = out_size, out_size * 2
            setattr(self, f'layer{idx}', layer)
            if idx == 1:
                downsampling = 2
                side = int(28 / 2 / downsampling / 2 ** (layers_count - 2))
                if side == 0:
                    side = 1
                in_feaures = side * side * 64 * 2 ** (layers_count - 2)
                # print([in_feaures, current_features])
                layer_function = nn.Linear(in_feaures, current_features)
            elif idx == layers_count:
                # print(current_features)
                layer_function = nn.Linear(current_features, 10)
            else:
                prev_layer = getattr(self, f'fc{idx - 1}')
                if idx < (self.layers_count // 2):
                    current_features *= 2
                else:
                    current_features //= 2
                # print([prev_layer.out_features, current_features])
                layer_function = nn.Linear(prev_layer.out_features, current_features)
            setattr(self, f'fc{idx}', layer_function)

    def forward(self, x):
        out = x
        idx = 0
        while idx < self.layers_count:
            idx += 1
            layer = getattr(self, f'layer{idx}')
            out = layer(out)
            # print(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        idx = 0
        while idx < self.layers_count:
            idx += 1
            fc = getattr(self, f'fc{idx}')
            out = fc(out)
        return out

In [213]:
conv_num_epochs = 1
conv_criterion = nn.CrossEntropyLoss()
conv_model = ConvNet(layers_count=2)
# conv_optimizer = torch.optim.Adam(conv_model.parameters(), lr=0.01)
conv_optimizer = optim.SGD(params=conv_model.parameters(), lr=0.05)

total_step = len(train_loader)
loss_list, acc_list, conv_epoch_losses = [], [], []

start_time = time.time()
for epoch in range(conv_num_epochs):
    conv_running_loss = 0.0
    conv_model.train()
    for i, (conv_images, conv_labels) in enumerate(train_loader):
        # Прямой запуск
        outputs = conv_model(conv_images)
        conv_loss = conv_criterion(outputs, conv_labels)
        loss_list.append(conv_loss.item())

        # Обратное распространение и оптимизатор
        conv_optimizer.zero_grad()
        conv_loss.backward()
        conv_optimizer.step()
        conv_running_loss += conv_loss.item() * conv_images.size(0)  # Учитываем потери по всем изображениям

        # Отслеживание точности
        total = conv_labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == conv_labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 300 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, conv_num_epochs, i + 1, total_step, conv_loss.item(), (correct / total) * 100))

    conv_epoch_losses.append(conv_running_loss / len(train_loader.dataset))  # Вычисляем среднюю потерю за эпоху
end_time = time.time()
print(f"Потери по эпохам:", conv_epoch_losses)
print("Время обучения Conv модели: {:.2f} секунд".format(end_time - start_time))

Epoch [1/1], Step [300/600], Loss: 0.0799, Accuracy: 99.00%
Epoch [1/1], Step [600/600], Loss: 0.1456, Accuracy: 94.00%
Потери по эпохам: [0.25180290155112744]
Время обучения Conv модели: 61.32 секунд


In [210]:
def assessment(model):
    model.eval()
    as_total, as_correct = 0, 0
    with torch.no_grad():
        for as_images, as_labels in test_loader:
            as_outputs = model(as_images)
            _, as_predicted = torch.max(as_outputs.data, 1)
            as_total += as_labels.size(0)
            as_correct += (as_predicted == as_labels).sum().item()

        as_accuracy = 100 * as_correct / as_total
    return as_accuracy

In [215]:
print(f'Точность FC модели при проверке на тестовом наборе: {assessment(fc_model):.2f}%')
print(f'Точность Conv модели при проверке на тестовом наборе: {assessment(conv_model):.2f}%')

Точность FC модели при проверке на тестовом наборе: 93.60%
Точность Conv модели при проверке на тестовом наборе: 97.32%


In [ ]:
 # torch.save(conv_model.state_dict(), 'conv_net_model.ckpt')